In [1]:
import numpy as np 
import tensorflow as tf
from tqdm import tqdm 

In [2]:
text = open('input.txt','rb').read()
text = str(text)

In [3]:
text = text.replace('\n',' ')
text = text.replace('\r',' ')
text = text.replace('\\n',' ')
text = text.replace('\\r',' ')
text = text.replace('.','')
text = text.lower()
text[:1000]

'b"first citizen:  before we proceed any further, hear me speak    all:  speak, speak    first citizen:  you are all resolved rather to die than to famish?    all:  resolved resolved    first citizen:  first, you know caius marcius is chief enemy to the people    all:  we know\'t, we know\'t    first citizen:  let us kill him, and we\'ll have corn at our own price  is\'t a verdict?    all:  no more talking on\'t; let it be done: away, away!    second citizen:  one word, good citizens    first citizen:  we are accounted poor citizens, the patricians good  what authority surfeits on would relieve us: if they  would yield us but the superfluity, while it were  wholesome, we might guess they relieved us humanely;  but they think we are too dear: the leanness that  afflicts us, the object of our misery, is as an  inventory to particularise their abundance; our  sufferance is a gain to them let us revenge this with  our pikes, ere we become rakes: for the gods know i  speak this in hunger fo

In [4]:
vocab = []
for i in  tqdm(text):
    if i not in vocab:
        vocab.append(i)
len(vocab)

100%|███████████████████████████████████████████████████████████████████| 1147512/1147512 [00:00<00:00, 2419875.69it/s]


38

In [5]:
text = text[0:100000]

In [6]:
chars = sorted(list(set(text)))
print('total chars: ', len(chars))

total chars:  36


In [7]:
chars

[' ',
 '!',
 '"',
 '&',
 "'",
 ',',
 '-',
 ':',
 ';',
 '?',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [8]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [9]:
len(text)

100000

In [10]:
max_len = 50
X_data = []
y = []
for i in tqdm(range(len(text)-max_len)):
    inp = text[i:i+max_len]
    out = text[i+max_len]
    X_data.append([char_indices[char] for char in inp ])
    y.append(char_indices[out])

100%|████████████████████████████████████████████████████████████████████████| 99950/99950 [00:00<00:00, 263697.63it/s]


In [11]:
from keras.utils import np_utils
X = np.array(X_data)
X = X.reshape((X.shape[0],50,1))
X = X/float(len(chars))
y = np_utils.to_categorical(y)
y = y.reshape((y.shape[0],y.shape[1]))

Using TensorFlow backend.


In [12]:
from keras.models import Sequential
from keras.layers import LSTM , Dense , Dropout
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='RMSprop')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 256)               264192    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 36)                9252      
Total params: 273,444
Trainable params: 273,444
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


In [ ]:
import sys
import numpy as np
from keras.models import load_model

model = load_model('model.h5')
def op(model):
    part = np.random.randint(0, len(X_data) - 1)
    data = X_data[part]
    print(''.join([indices_char[val] for val in data]))
    for i in range(1000):
        x = np.array(data)
        x = x.reshape((1,len(data),1))
        x = x/float(len(chars))
        pred = model.predict(x,verbose=0)[0]
        idx = sample(pred)
        res = indices_char[idx]
        sys.stdout.write(res)
        data.append(idx)
        data = data[1:len(data)]
        
op(model)

In [30]:
for i in range(20):
    model.fit(X,y,batch_size=128,epochs=1)
    op(model)

Epoch 1/1
99950/99950 [==============================] - 35s 352us/step - loss: 2.2433
ilder? would you have me  false to my nature? rath
ehs arm gamtr  ngres absotd woued heands    bimio anta,le shruln    aoiinius:  ha hase  eont:    menenius:  miwe hhrne uwunfve    vecond ci ioe:ui tel,: fodp    paccnt :ipnge::    mininius:  bor goay aaro c shrtkr  teo tiln bere-reeuh 'wh maso hitdelue yiuc shey;  taani h taa botovrd '  f mrnil  a paatesror ctr oi mim,    oonsni citizen:  shrrshk no a cvte ons?m-big yhuh thado  ea dmcsd hgnedr to poer 'ia celcn for go tetlnn  fuew i  rho tl tftn yhohheh, whar wsu hrsr thltr,ad-  whoh they cnsutrou but tle par,net:toluue  thttgd mi;  yhe heo hu hh  a lrna enei  ik miltelte nhri hormoug?c 'uhs siey sahe     vulumnio:  b waidie shan r  aof ta roesl thiy beecoe sh the mfohlet;  otei sntep fnv thet  aldmcn tueksiek thous ilrioa bndb     moriniasus:  a;meo tyr them pu naa ie hapr tteuh;    mrreniasus:  nh,soese fo ito al' s,eiir oo my mhrcer  py sovums fo 

KeyboardInterrupt: 

In [15]:
model.save('model.h5')

 a traitorous innovator,  a foe to the public weal


 every  thing hereafter    volumnia:  let her alon
,,aorl an yhu, teev wou  a aoptue uf cltonues sr ayridm?hs    birizens:  sot aeoig lo bidi    sicrnd citizen:  wuelu ofiy,o modati,  whoo touu,o thl ba tpnnon  b oaiher  n pieaedr ohr,li,n mooran da fitdt wfrn thr hav ,admse boire  whet nhnegee shay aus wuurr    vecond cptizen:  yeenh ntw

KeyboardInterrupt: 